In [21]:
 PROJECT_ID = "cap-gen-app-builder-infobot" 
# gcloud config set project {PROJECT_ID}

In [22]:
import random
import string
from google.cloud import aiplatform

# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()

In [23]:
BUCKET_NAME = "genai_telecom_true_nps"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"  # @param {type: "string"}

In [24]:
from typing import Union
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from vertexai.preview.language_models import TextGenerationModel
from google.cloud import aiplatform
from google.cloud import bigquery

In [25]:
train = pd.read_csv('output_21_9_v2.csv')
train

,Sentiment,Product,Company_Name,Category,Country,State,Input_tweet
0,negative,Galaxy S,Samsung,Mobile,United States,Texas,Excited to meet the @samsungmobileus at #sxsw ...
1,positive,android,Google,Technology,United States,Texas,This is a #WINNING picture #android #google #s...
2,Negative,Lego,Lego,Toys,United States,Texas,they took away the lego pit but replaced it wi...
3,positive,Android,Google,Computers,United States,Texas,Alert the media. I just saw the one and only ...
4,positive,iOS,Apple,Mobile,United States,Texas,"Farooqui: Now about mobile. iOS, with Android ..."
...,...,...,...,...,...,...,...
886,positive,Optiscan,Optiscan,Software,United States,Texas,Any #SXSW attendees need a QR code reader for ...
887,Positive,iPhone,Apple,Technology,United States,Texas,{link} Go beyond Earth Hour this year by shari...
888,Positive,iPad,RSQ,App,United States,Texas,Nice find by RSQ's @mention {link} #sxsw #iPad...
889,positive,Holler Gram,Holler Gram,App,United States,Texas,"Great for certain meetings. RT @mention Hello,..."


In [39]:
print(train)

    Sentiment      Product Company_Name    Category        Country  State  \
0    negative     Galaxy S      Samsung      Mobile  United States  Texas   
1    positive      android       Google  Technology  United States  Texas   
2    Negative         Lego         Lego        Toys  United States  Texas   
3    positive      Android       Google   Computers  United States  Texas   
4    positive          iOS        Apple      Mobile  United States  Texas   
..        ...          ...          ...         ...            ...    ...   
886  positive     Optiscan     Optiscan    Software  United States  Texas   
887  Positive       iPhone        Apple  Technology  United States  Texas   
888  Positive         iPad          RSQ         App  United States  Texas   
889  positive  Holler Gram  Holler Gram         App  United States  Texas   
890  Positive      SXSW Go         SXSW       Event  United States  Texas   

                                                 input  \
0    Excited to m

In [44]:
train = train.rename(columns={'Input_tweet': 'input_text'})

In [45]:
train = train.rename(columns={'output_text': 'output_text'})

In [46]:

# train = train.dropna()
train['output_text']=''
for i in range(len(train)):
    train[str('output_text')][i] = 'Sentiment-'+ str(train['Sentiment'][i]) + ',Product-'+ str(train['Product'][i])+ ',Company_Name-'+ str(train['Company_Name'][i])+ ',Category-'+ str(train['Category'][i]) + ',Country-'+ str(train['Country'][i]) +',State-'+ str(train['State'][i])

In [47]:
train_df = train[['input_text','output_text']]
train_df

KeyError: "['input_text'] not in index"

In [ ]:
tune_jsonl = train_df.to_json(orient='records', lines=True)

print(f"Length: {len(tune_jsonl)}")
print(tune_jsonl[0:100])

In [ ]:
training_data_filename = "tune_data_27_9_v1.jsonl"

with open(training_data_filename, "w+") as f:
    f.write(tune_jsonl)

In [2]:
import pandas as pd

In [3]:
t_df = pd.read_csv('output_21_9_v2.csv')
t_df

,Sentiment,Product,Company_Name,Category,Country,State,Input_tweet
0,negative,Galaxy S,Samsung,Mobile,United States,Texas,Excited to meet the @samsungmobileus at #sxsw ...
1,positive,android,Google,Technology,United States,Texas,This is a #WINNING picture #android #google #s...
2,Negative,Lego,Lego,Toys,United States,Texas,they took away the lego pit but replaced it wi...
3,positive,Android,Google,Computers,United States,Texas,Alert the media. I just saw the one and only ...
4,positive,iOS,Apple,Mobile,United States,Texas,"Farooqui: Now about mobile. iOS, with Android ..."
...,...,...,...,...,...,...,...
886,positive,Optiscan,Optiscan,Software,United States,Texas,Any #SXSW attendees need a QR code reader for ...
887,Positive,iPhone,Apple,Technology,United States,Texas,{link} Go beyond Earth Hour this year by shari...
888,Positive,iPad,RSQ,App,United States,Texas,Nice find by RSQ's @mention {link} #sxsw #iPad...
889,positive,Holler Gram,Holler Gram,App,United States,Texas,"Great for certain meetings. RT @mention Hello,..."


In [5]:
# train = train.dropna()
t_df['output_text']=''
for i in range(len(t_df)):
    t_df[str('output_text')][i] = 'Sentiment-'+ str(t_df['Sentiment'][i]) + '|Product-'+ str(t_df['Product'][i])+ '|Company_Name-'+ str(t_df['Company_Name'][i])+ '|Category-'+ str(t_df['Category'][i]) + '|Country-'+ str(t_df['Country'][i]) +'|State-'+ str(t_df['State'][i])

In [6]:
t_df=t_df[['Input_tweet', 'output_text']]

In [7]:
t_df

,Input_tweet,output_text
0,Excited to meet the @samsungmobileus at #sxsw ...,Sentiment-negative|Product-Galaxy S|Company_Na...
1,This is a #WINNING picture #android #google #s...,Sentiment-positive|Product-android|Company_Nam...
2,they took away the lego pit but replaced it wi...,Sentiment-Negative|Product-Lego|Company_Name-L...
3,Alert the media. I just saw the one and only ...,Sentiment-positive|Product-Android|Company_Nam...
4,"Farooqui: Now about mobile. iOS, with Android ...",Sentiment-positive|Product-iOS|Company_Name-Ap...
...,...,...
886,Any #SXSW attendees need a QR code reader for ...,Sentiment-positive|Product-Optiscan|Company_Na...
887,{link} Go beyond Earth Hour this year by shari...,Sentiment-Positive|Product-iPhone|Company_Name...
888,Nice find by RSQ's @mention {link} #sxsw #iPad...,Sentiment-Positive|Product-iPad|Company_Name-R...
889,"Great for certain meetings. RT @mention Hello,...",Sentiment-positive|Product-Holler Gram|Company...


In [ ]:
column_array = {'Input_tweet': train['Input_tweet'].to_numpy(), 'output_text': train['output_text'].to_numpy() }


In [16]:
columns_to_convert = ['Input_tweet','output_text']
column_arrays = {}
for column_name in columns_to_convert:
    # Convert each column to an array and store it in the dictionary
    column_arrays[column_name] = t_df[column_name].to_numpy()
    print(column_arrays)

{'Input_tweet': array(['Excited to meet the @samsungmobileus at #sxsw so I can show them my Sprint Galaxy S still running Android 2.1.   #fail',
       'This is a #WINNING picture #android #google #sxsw {link}',
       'they took away the lego pit but replaced it with a recharging station ;) #sxsw and i might check prices for an iphone - crap samsung android',
       'Alert the media.  I just saw the one and only Android tablet at #sxsw.  Like finding a needle in a haystack!  I also saw a Cr-48.',
       'Farooqui: Now about mobile. iOS, with Android catching up fast and will grow more once they allow in-app purchasing. #gamesfortv #sxsw',
       'Talked to some great developers at the Android meetup. Looking forward to working with them. #sxsw #android #androidsxsw',
       "There are thousands of iPad 2's floating around Austin at #sxsw and I have not seen even one single Android tablet.  Not even one. Zero.",
       'Woot! RT @mention First Android @mention disc {link} ... Market ve

In [19]:
my_list = column_arrays.tolist()

AttributeError: 'dict' object has no attribute 'tolist'

In [18]:
json_string = json.dumps(my_list)

TypeError: Object of type ndarray is not JSON serializable

In [17]:
tune_jsonl = json_string.to_json(orient='records', lines=True)

AttributeError: 'dict' object has no attribute 'to_json'